<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

# TinyLLama Few-Shot learning

In diesem Notebook werden wir einen Chatbot für Schweizer Immobilien Empfehlungen mittels Few-Shot learning erstellen. <br>
Dabei verwenden wir das LLM [TinyLLama chat](https://huggingface.co/TinyLlama/TinyLlama-1.1B-step-50K-105b) von Meta.



---
Bearbeitet durch Si Ben Tran, Yannic Lais, Rami Tarabishi im HS 2023.<br>
Bachelor of Science FHNW in Data Science.

## Einleitung

### Allgemeines Vorgehen

- Name entity recognition auf den Prompt
- Entities werden für die Datenbankabfrage extrahiert
- Prompt wird mit den Trainingsexamples sowie der Datenbankabfrage an das TinyLlama Modell gesendet

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re

C:\Users\rami0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## Immobiliendaten

In [3]:
# read parquet file
df = pd.read_parquet('data\immo_data_202208.parquet')

In [4]:
# filter for important columns
df = df[['Municipality', 'detailed_description', 'price_cleaned', 'type', 'rooms', 'url']]
df['price'] = df['price_cleaned'].astype(float)
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce')
df = df.drop(columns=['price_cleaned'])
# drop rows with missing values
df = df.dropna()

In [6]:
df.head()

,Municipality,detailed_description,type,rooms,url,price
0,Biberstein,DescriptionLuxuriöse Attika-Wohnung direkt an ...,penthouse,5.0,https://www.immoscout24.ch//en/d/penthouse-buy...,1150000.0
1,Biberstein,DescriptionStilvolle Liegenschaft an ruhiger L...,terrace-house,5.0,https://www.immoscout24.ch//en/d/terrace-house...,1420000.0
3,Biberstein,DescriptionDieses äusserst grosszügige Minergi...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,1430000.0
4,Küttigen,DescriptionAus ehemals zwei Wohnungen wurde ei...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-romb...,995000.0
5,Erlinsbach (AG),DescriptionDer Blick in die Weite vermittelt R...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,2160000.0


## TimyLlama Modell

In [5]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="cuda", trust_remote_code=True)
model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

#### Einzelner Input

In [6]:
inputs = tokenizer("Is a penguin a bird or a mamal?", return_tensors="pt").to('cuda')

# Generate outputs and decode
outputs = model.generate(**inputs, max_length=40)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

Is a penguin a bird or a mamal?


## Process Prompt

In [7]:
# Few-shot examples
few_shot_examples = [
    {
        "Question": "I am looking for a flat in Zurich under 1'000'000 CHF.", 
        "Answer": "Here are some options for apartments in Zurich under 1'000'000 CHF: (max_price = 1000000, location_keyword = Zürich, property_type = flat)"
    },
    {
        "Question": "Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?",
        "Answer": "Yes, there are terraced houses in Bern in the CHF 500,000 to 700,000 range: (max_price = 700000, min_price = 500000, location_keyword = Bern, property_type = terraced_house)"
    },
    {
        "Question": "I need a detached house in Lucerne with a garden for around CHF 1,200,000.",
        "Answer": "In Lucerne you can find detached houses with a garden for around CHF 1,200,000: (location_keyword = Bern, property_type = detached-house, arround_price = 1200000)"
    },
    {
        "Question": "Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?",
        "Answer": "Modern apartments in Basel under 900'000 CHF are available: (max_price = 900000, location_keyword = Basel, property_type = flat, rooms = 3.5)"
    },
    {
        "Question": "I am looking for a large house in Lausanne, at least 5 rooms, up to 1'500'000 CHF.",
        "Answer": "Large houses in Lausanne with at least 5 rooms up to 1'500'000 CHF can be found here: (max_price = 1500000, location_keyword = Lausanne, property_type = house)"
    },
    {
        "Question": "I am looking for a house in Geneva with at least 4 rooms and a garden.",
        "Answer": "Here are some options for houses in Geneva with at least 4 rooms and a garden: (location_keyword = Geneva, property_type = house, rooms = 4)"
    }

]


In [8]:
def filter_dataframe(df = df, max_price = None, min_price = None, arround_price = None, location_keyword = None, property_type = None, rooms = None):

    type_list =  df.type.unique().tolist()
    type_list.append('house')
    type_list = [x.lower() for x in type_list]

    # Apply filters
    if arround_price:
        filtered_df = df[df['price'] <= arround_price * 1.1]
        filtered_df = df[df['price'] >= arround_price * 0.9]
    if max_price:
        filtered_df = df[df['price'] <= max_price]
    if min_price:
        filtered_df = df[df['price'] >= min_price]
    if location_keyword:
        filtered_df = filtered_df[filtered_df['Municipality'].str.contains(location_keyword, case=False, na=False)]
    if property_type.lower() in type_list:
        filtered_df = filtered_df[filtered_df['type'].str.contains(property_type, case=False, na=False)]
    if rooms:
        filtered_df = filtered_df[filtered_df['rooms'] == rooms]

    # Return 5 random samples
    if len(filtered_df) >= 5:
        return filtered_df.sample(n=5, random_state = 42)
    else:
        return filtered_df

In [9]:
test = filter_dataframe(df, max_price = 1200000, location_keyword = 'Basel', property_type = 'flat')

In [10]:
test.head()

,Municipality,detailed_description,type,rooms,url,price
2742,Basel,DescriptionObjektbeschrieb An idealer Lage im...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,495000.0
2738,Basel,DescriptionDiese Wohnung im 3. Obergeschoss bi...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,650000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2737,Basel,DescriptionIm Breite-Quartier wird diese heime...,flat,2.0,https://www.immoscout24.ch//en/d/flat-buy-base...,470000.0
2733,Basel,DescriptionBaujahr: 1961Renovation: 2022Wohnfl...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,695000.0


In [17]:
def get_model_response(query, model, tokenizer, few_shots = None, pre_query = None):
    # Load model and tokenizer
    model = model
    tokenizer = tokenizer

    if few_shots:
        prompt_text = "\n".join([f"Question: {ex['Question']}\nAnswer: {ex['Answer']}" for ex in few_shots])
        prompt_text += f"\nQuestion: {query}"
        prompt_text += "\nAnswer:"
    elif pre_query:
        prompt_text = pre_query
        prompt_text += f"\nQuestion: {query}"
        prompt_text += "\nAnswer:"
    else:
        prompt_text = f"Question: {query}\nAnswer:"

    # Encode and send to model
    inputs = tokenizer(prompt_text, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs, max_length=1024, num_return_sequences=1)

    # Decode the output
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extracting the answer corresponding to the specific query
    response_parts = full_response.split("Answer:")
    for i, part in enumerate(response_parts[:-1]):
        if f"Question: {query}" in part:
            return response_parts[i + 1].split("\n")[0].strip()

    return "No specific answer found."

In [24]:
def parse_and_filter(input_str, df):
    try:
        # Extract the parameter string
        params_str = re.search(r'\((.*?)\)', input_str).group(1)
    except:
        return "No specific answer found."

    # Initialize parameters with default values
    params = {
        'df': df,
        'max_price': None,
        'min_price': None,
        'arround_price': None,
        'location_keyword': None,
        'property_type': None,
        'rooms': None
    }

    print(params_str)

    # Split the parameter string and iterate over each parameter
    for param in params_str.split(','):
        key, value = param.split('=')
        key = key.strip()
        value = value.strip()

        # Convert value to the correct type if necessary
        if key in ['max_price', 'min_price', 'arround_price', 'rooms']:
            value = int(value)
        # Update the parameters dictionary
        params[key] = value

    # Call the filter_dataframe function with unpacked arguments
    return filter_dataframe(**params)

#### Few-shot learning:

In [19]:
# Example usage
query = "Give me houses in aarau which cost less than 4'000'000 CHF."
response = get_model_response(query, model=model, tokenizer=tokenizer, few_shots=few_shot_examples)
print(response)

filtered_df = parse_and_filter(response, df)

filtered_df

Question: I am looking for a flat in Zurich under 1'000'000 CHF.
Answer: Here are some options for apartments in Zurich under 1'000'000 CHF: (max_price = 1000000, location_keyword = Zürich, property_type = flat)
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: Yes, there are terraced houses in Bern in the CHF 500,000 to 700,000 range: (max_price = 700000, min_price = 500000, location_keyword = Bern, property_type = terraced_house)
Question: I need a detached house in Lucerne with a garden for around CHF 1,200,000.
Answer: In Lucerne you can find detached houses with a garden for around CHF 1,200,000: (location_keyword = Bern, property_type = detached-house, arround_price = 1200000)
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: Modern apartments in Basel under 900'000 CHF are available: (max_price = 900000, location_keyword = Basel, property_type = flat, rooms = 3.5)
Question: I am looking f

,Municipality,detailed_description,type,rooms,url,price
12,Aarau,DescriptionDas im Jahr 1936 massiv in Beton er...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,1200000.0
167,Aarau,DescriptionVom Bahnhof Aarau brauchen Sie ledi...,semi-detached-house,5.0,https://www.immoscout24.ch//en/d/semi-detached...,2090000.0
16,Aarau,DescriptionWohnen im Grünen und doch in der St...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,695000.0
11,Aarau,DescriptionDiese interessante 4 ½ Zimmer-Attik...,penthouse,5.0,https://www.immoscout24.ch//en/d/penthouse-buy...,845000.0
70,Aarau,DescriptionATEMBERAUBENDE 5 1/2- Zimmerwohnung...,penthouse,5.0,https://www.immoscout24.ch//en/d/penthouse-buy...,1890000.0


TinyLlama versteht den prompt perfekt und gibt züruck, was wir erwarten.

#### Few-shot mit einem deutchen prompt:

In [20]:
# Example usage
query_german = "Zeige mir ein paar Wohnungen in Zürich, die umgefär 2'500'000 CHF kosten."
response = get_model_response(query_german, model=model, tokenizer=tokenizer, few_shots=few_shot_examples)
print(response)

filtered_df = parse_and_filter(response, df)

filtered_df

Question: I am looking for a flat in Zurich under 1'000'000 CHF.
Answer: Here are some options for apartments in Zurich under 1'000'000 CHF: (max_price = 1000000, location_keyword = Zürich, property_type = flat)
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: Yes, there are terraced houses in Bern in the CHF 500,000 to 700,000 range: (max_price = 700000, min_price = 500000, location_keyword = Bern, property_type = terraced_house)
Question: I need a detached house in Lucerne with a garden for around CHF 1,200,000.
Answer: In Lucerne you can find detached houses with a garden for around CHF 1,200,000: (location_keyword = Bern, property_type = detached-house, arround_price = 1200000)
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: Modern apartments in Basel under 900'000 CHF are available: (max_price = 900000, location_keyword = Basel, property_type = flat, rooms = 3.5)
Question: I am looking f

,Municipality,detailed_description,type,rooms,url,price


TinyLlama versteht und antwortet auch den deutchen prompt korrekt und antwortet sogar auf deutsch.

### Random query ohne few shot:

In [21]:
query_random = "Hello do you have bread?"

response = get_model_response(query_random, model=model, tokenizer=tokenizer)
filtered_df = parse_and_filter(response, df)

print(response.split(':')[0])
filtered_df

Question: Hello do you have bread?
Answer:
Yes, we have bread.


'No specific answer found.'

In [25]:
# Mit fewshot
query_random = "Hello do you have bread?"

response = get_model_response(query_random, model=model, tokenizer=tokenizer, few_shots=few_shot_examples)
filtered_df = parse_and_filter(response, df)

print(response)
filtered_df

Question: I am looking for a flat in Zurich under 1'000'000 CHF.
Answer: Here are some options for apartments in Zurich under 1'000'000 CHF: (max_price = 1000000, location_keyword = Zürich, property_type = flat)
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: Yes, there are terraced houses in Bern in the CHF 500,000 to 700,000 range: (max_price = 700000, min_price = 500000, location_keyword = Bern, property_type = terraced_house)
Question: I need a detached house in Lucerne with a garden for around CHF 1,200,000.
Answer: In Lucerne you can find detached houses with a garden for around CHF 1,200,000: (location_keyword = Bern, property_type = detached-house, arround_price = 1200000)
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: Modern apartments in Basel under 900'000 CHF are available: (max_price = 900000, location_keyword = Basel, property_type = flat, rooms = 3.5)
Question: I am looking f

ValueError: not enough values to unpack (expected 2, got 1)

Die few shot beispiele shienen das model zu verwirren und es probiert die random query an den gleichen vormat den beispiele zu bringen mit eine gemeinde, typ von immobilien die eine bäkerei ist, und danach brot.

### Extrahieren des gesammten Dataframe Abfrage Codes

In [26]:
few_shot_examples_code = [
    {
        "Question": "I am looking for an flat in Zurich under 1'000'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('flat'))]"
    },
    {
        "Question": "Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]"
    },
    {
        "Question": "I need a house in Lucerne with a garden for around CHF 1,200,000.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] <= (1200000*1.1)) & (df['price'] >= (1200000*0.9)) & (df['type'].str.contains('house'))]"
    },
    {
        "Question": "Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment'))]"
    },
    {
        "Question": "I am looking for a large house in Lausanne, at least 5 rooms, up to 1'500'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Lausanne')) & (df['rooms'] >= 5) & (df['price'] <= 1500000) & (df['type'].str.contains('house'))]"
    }
]

In [27]:
query = "Show me flats in Basel which cost less than 2000000 CHF with 3 or more rooms."

response = get_model_response(query, model=model, tokenizer=tokenizer, few_shots=few_shot_examples_code)

print(response)

exec(response)
df_test.head()

Question: I am looking for an flat in Zurich under 1'000'000 CHF.
Answer: df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('flat'))]
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]
Question: I need a house in Lucerne with a garden for around CHF 1,200,000.
Answer: df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] <= (1200000*1.1)) & (df['price'] >= (1200000*0.9)) & (df['type'].str.contains('house'))]
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment'))]
Question: I am looking for a large house in Lausanne, at least 5 room

,Municipality,detailed_description,type,rooms,url,price


In [31]:
df_test = df[(df['Municipality'].str.contains('Basel')) & (df['price'] < 2000000) & (df['rooms'] >= 3) & (df['type'].str.contains('flat'))]

df_test.head()

,Municipality,detailed_description,type,rooms,url,price
2670,Basel,"DescriptionGrosszügige, sonnige 3.5-Zimmerwohn...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1450000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2675,Basel,"DescriptionDie helle, charmante 4.5 Zimmerwohn...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1280000.0
2705,Basel,DescriptionDie moderne 3-Zimmer Wohnung befind...,flat,3.0,https://www.immoscout24.ch//en/d/flat-buy-base...,670000.0
2709,Basel,DescriptionDiese helle und geräumige Etagenwoh...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1380000.0


Die antwort ist eigendlich richtig, aber aus irgendeinem grund hat es 'apartment' als immobilien typ genommen wenn ich explicit 'flats' in der query gebraucht habe. Mann kann auch sagen das dies problem nicht vom model kommt sondern unsere daten, aber an der gleiche zeit hatte es 'flat' solten verstehen von meiner frage. Das kann man auch anpassen mit bessere few shots die ich danach unten angebe.

In [33]:
query = "I need a house in Basel with a garden for around CHF 1200000."

response = get_model_response(query, model=model, tokenizer=tokenizer, few_shots=few_shot_examples_code)

print(response)

exec(response)
df_test.head()

Question: I am looking for an flat in Zurich under 1'000'000 CHF.
Answer: df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('flat'))]
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]
Question: I need a house in Lucerne with a garden for around CHF 1,200,000.
Answer: df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] <= (1200000*1.1)) & (df['price'] >= (1200000*0.9)) & (df['type'].str.contains('house'))]
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment'))]
Question: I am looking for a large house in Lausanne, at least 5 room

,Municipality,detailed_description,type,rooms,url,price
2701,Basel,DescriptionNur wenige Schritte vom Rhein entfe...,terrace-house,7.0,https://www.immoscout24.ch//en/d/terrace-house...,1290000.0
2715,Basel,DescriptionAn einer ruhigen Nebenstrasse in de...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,1250000.0


In [32]:
query = "I am looking for a house in Biberstein which costs about 1000000."

response = get_model_response(query, model=model, tokenizer=tokenizer, few_shots=few_shot_examples_code)

print(response)

exec(response)
df_test.head()

Question: I am looking for an flat in Zurich under 1'000'000 CHF.
Answer: df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('flat'))]
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]
Question: I need a house in Lucerne with a garden for around CHF 1,200,000.
Answer: df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] <= (1200000*1.1)) & (df['price'] >= (1200000*0.9)) & (df['type'].str.contains('house'))]
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment'))]
Question: I am looking for a large house in Lausanne, at least 5 room

,Municipality,detailed_description,type,rooms,url,price
6,Biberstein,DescriptionZum Objekt:Kompakt und doch sehr ge...,terrace-house,5.0,https://www.immoscout24.ch//en/d/terrace-house...,550000.0


Hier hat tinyllama besser gemacht als Phi-2, da es nicht ein genauen preis genommen hat, sondern under 1 million. Theoretisch hat es sollen den 'about' wert nehmen der +- 10% von 1 million ist. Aber immer noch besser als den genauen preis.

### Query 5: Test test mit multiplen typen

In [34]:
few_shot_examples_multiple_type_code = [
    {
        "Question": "I am looking for an flat in Zurich under 1'000'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]"
    },
    {
        "Question": "Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]"
    },
    {
        "Question": "I need a detached house in Lucerne with a garden for around CHF 1,200,000.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] == 1200000) & (df['type'].str.contains('house'))]"
    },
    {
        "Question": "Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]"
    },
    {
        "Question": "I am looking for a large house in Lausanne, at least 5 rooms, up to 1'500'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Lausanne')) & (df['rooms'] >= 5) & (df['price'] <= 1500000) & (df['type'].str.contains('house'))]"
    }
]

In [36]:
query = "Show me apartments in Bern for less than 1'000'000 CHF."

response = get_model_response(query, model=model, tokenizer=tokenizer, few_shots=few_shot_examples_multiple_type_code)

print(response)

exec(response)
df_test.head()

Question: I am looking for an flat in Zurich under 1'000'000 CHF.
Answer: df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]
Question: I need a detached house in Lucerne with a garden for around CHF 1,200,000.
Answer: df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] == 1200000) & (df['type'].str.contains('house'))]
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]
Question: I am looking for a large house i

,Municipality,detailed_description,type,rooms,url,price
1728,Bern,DescriptionIdéalement situé dans le quartier d...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,680000.0
1735,Bern,DescriptionZentrumsnahes Wohnen in der Stadt B...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,555000.0
1741,Bern,DescriptionSchöne sonnige Wohnung auf 3.en Eta...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,351250.0
1745,Bern,DescriptionEin Herz für das Ostring-Quartier!A...,flat,7.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,985000.0
1750,Bern,DescriptionLageDas Objekt befindet sich in ein...,attic-flat,5.0,https://www.immoscout24.ch//en/d/attic-flat-bu...,930000.0


In [38]:
# Retry the example tinyllama got the apartment wrong
query = "Show me flats in Basel which cost less than 2000000 CHF with 3 or more rooms."

response = get_model_response(query, model=model, tokenizer=tokenizer, few_shots=few_shot_examples_multiple_type_code)

print(response)

exec(response)
df_test.head()

Question: I am looking for an flat in Zurich under 1'000'000 CHF.
Answer: df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]
Question: Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?
Answer: df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]
Question: I need a detached house in Lucerne with a garden for around CHF 1,200,000.
Answer: df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] == 1200000) & (df['type'].str.contains('house'))]
Question: Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?
Answer: df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]
Question: I am looking for a large house i

,Municipality,detailed_description,type,rooms,url,price
2670,Basel,"DescriptionGrosszügige, sonnige 3.5-Zimmerwohn...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1450000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2675,Basel,"DescriptionDie helle, charmante 4.5 Zimmerwohn...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1280000.0
2699,Basel,DescriptionDiese wunderschöne Attikawohnung li...,penthouse,5.0,https://www.immoscout24.ch//en/d/penthouse-buy...,1590000.0
2705,Basel,DescriptionDie moderne 3-Zimmer Wohnung befind...,flat,3.0,https://www.immoscout24.ch//en/d/flat-buy-base...,670000.0


Jetzt mit den neuen few shots die mehere typen aus geben, hat tinyllama kein problem mit den typ falsch ausgeben.

## Fazit

Da tinyllama wie Pi-2 auch ein grösseres modell ist, überrascht es nicht, das es die meisten Abfragen beim ersten mal richtig gekriegt hat. Das Einzige, was micht überraschte, war die Random query mit few shots. Tinyllama scheint ein sehr "obedient" modell zu sein und nimmt den promt sehr wörtlich, indem es auch probiert die richtige Parameter zurückgeben wie die beispiele, es hat gedacht ich suche nach eine bäckerei.

Im ende, wenn man es mit Phi-2 vergleicht, hat es queries richtig gekriegt die Phi-2 falsch gekreigt hat.